Importing relevant tools

In [ ]:
from lpCompiler import _blocks
from lpModels import modelShell

In [ ]:
class mSimple(modelShell):
	""" This class includes 
		(1) An electricity market, 
		(2) Electricity storage """
	def __init__(self, db, blocks = None, **kwargs):
		db.updateAlias(alias=[(k, k+'_constr') for k in ('h','id')])
		super().__init__(db, blocks=blocks, **kwargs)
		
	@property
	def globalDomains(self):
		return {'Generation': adj.rc_pd(pd.MultiIndex.from_product([self.db['h'], self.db['id']]),getTechs(['Standard'],self.db)),
				'discharge' : adj.rc_pd(pd.MultiIndex.from_product([self.db['h'], self.db['id']]),getTechs(['Storage'],self.db)),
				'charge'	: adj.rc_pd(pd.MultiIndex.from_product([self.db['h'], self.db['id']]),getTechs(['Storage'],self.db)),
				'stored'	: adj.rc_pd(pd.MultiIndex.from_product([self.db['h'], self.db['id']]),getTechs(['Storage'],self.db)),			
				'HourlyDemand': pyDbs.cartesianProductIndex([self.db['c'], self.db['h']]),
				'equilibrium': self.db['h_constr'],
				'LawOfMotion': pyDbs.cartesianProductIndex([adj.rc_AdjPd(getTechs('Storage',self.db), alias = {'id':'id_constr'}), self.db['h_constr']])}

	def initBlocks(self, **kwargs):
		[getattr(self.blocks, f'add_{t}')(**v) for t in _blocks if hasattr(self,t) for v in getattr(self,t)];
#From here the relevant vectors for the augmented linear programming problem is defined.
#We see that the c vector, i.e., the vector containing the coefficients of the choice variables include four decision variables.
	@property
	def c(self):
		return [{'varName': 'Generation', 'value': adjMultiIndex.bc(self.db['mc'], self.globalDomains['Generation']),'conditions': getTechs(['Standard'],self.db)},
				{'varName': 'HourlyDemand','value':-adjMultiIndex.bc(self.db['MWP'], self.globalDomains['HourlyDemand'])},
				{'varName': 'discharge', 'value': adjMultiIndex.bc(self.db['mc'], self.globalDomains['discharge']),'conditions': getTechs('Storage',self.db)},
				{'varName': 'charge','value': adjMultiIndex.bc(self.db['mc'], self.globalDomains['charge']),'conditions': getTechs('Storage',self.db)}]

	@property
	def u(self):
		return [{'varName': 'Generation', 'value': adjMultiIndex.bc(self.hourlyGeneratingCap, self.globalDomains['Generation']), 'conditions': getTechs(['Standard'],self.db)},
				{'varName': 'HourlyDemand', 'value': self.hourlyLoad_c},
				{'varName': 'stored', 'value': adjMultiIndex.bc(self.db['sCap'], self.globalDomains['stored'])},
				{'varName': 'discharge', 'value': adjMultiIndex.bc(self.db['GeneratingCapacity'], self.globalDomains['discharge']), 'conditions': getTechs('Storage',self.db)},
				{'varName': 'charge', 'value': adjMultiIndex.bc(self.db['GeneratingCapacity'], self.globalDomains['charge']), 'conditions': getTechs('Storage',self.db)}]
	
	@property
	def b_eq(self):
		return [{'constrName': 'equilibrium'},{'constrName':'LawOfMotion'}]
	
	@property
	def A_eq(self):
		return [{'constrName': 'equilibrium', 'varName': 'Generation', 'value': appIndexWithCopySeries(pd.Series(1, index = self.globalDomains['Generation']), 'h','h_constr')},
				{'constrName': 'equilibrium', 'varName': 'HourlyDemand', 'value': appIndexWithCopySeries(pd.Series(-1, index = self.globalDomains['HourlyDemand']), 'h','h_constr')},
				{'constrName': 'equilibrium', 'varName': 'discharge', 'value': appIndexWithCopySeries(pd.Series(1, index = self.globalDomains['discharge']), 'h', 'h_constr')},
				{'constrName': 'equilibrium', 'varName': 'charge', 'value': appIndexWithCopySeries(pd.Series(-1, index = self.globalDomains['charge']), 'h', 'h_constr')},
				{'constrName': 'LawOfMotion', 'varName': 'stored', 'value': [appIndexWithCopySeries(pd.Series(1, index = self.globalDomains['stored']), ['id','h'], ['id_constr','h_constr']),
																				 rollLevelS(appIndexWithCopySeries(adjMultiIndex.bc(-1, self.globalDomains['stored']), ['id','h'], ['id_constr','h_constr']), 'h',1)]},
				{'constrName': 'LawOfMotion', 'varName': 'discharge', 'value': appIndexWithCopySeries(adjMultiIndex.bc(1/self.db['effS'], self.globalDomains['stored']), ['id','h'], ['id_constr','h_constr']), 'conditions': getTechs('Storage',self.db)},
				{'constrName': 'LawOfMotion', 'varName': 'charge', 'value': appIndexWithCopySeries(adjMultiIndex.bc(-self.db['effS'] , self.globalDomains['stored']), ['id','h'], ['id_constr','h_constr']), 'conditions': getTechs('Storage',self.db)}]
	